In [64]:
#importing modules
from math import *
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 21})
import scipy.stats as stats
from scipy.optimize import curve_fit,leastsq
from scipy.integrate import odeint, ode
from scipy.interpolate import interp1d
import numpy as np
import pandas
from scipy.interpolate import interp1d

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# Södermanland	
# Sthlm

In [65]:
#Parameters
n=20
startvecka=10-4 #start week
Tmax=48 #length of prediction
pv=0.023 #fraction of infected that are admitted

rho=7*1/5.1 #latency period
gamma=7*1/5 #recovery rate

delay=3#2.5 #delay from infection to admission

In [66]:
file = pandas.read_excel('../data/statistik-covid19-inskrivna_v2.xlsx',sheet_name="Inskrivna i slutenvård")
data = [['Sthlm', 8,2389923], ['VGR', 9,1725881], ['Skåne', 10,1387650],['Östergötland',11,467095],['Jönköping',12,364750],['Södermanland',13,299101],['Uppsala',14,387628],['Gävleborg',15,287660],['Västmanland',16,277074],['Örebro',17,305726],['Norrbotten',18,249768],['Dalarna',19,287806],['Västernorrland',20,244855],['Värmland',21,282840],['Halland',22,336132],['Kalmar',23,245992],['Kronoberg',24,202163],['Västerbotten',25,273061],['Jämtland',26,130972],['Blekinge',27,159349],['Gotland',28,60106]] 
regioner = pandas.DataFrame(data, columns = ['region', 'index','population']) 
regionkod=['SE-AB','SE-O','SE-M','SE-E','SE-F','SE-D','SE-C','SE-X','SE-U','SE-T','SE-BD','SE-W','SE-Y','SE-S','SE-N','SE-H','SE-G','SE-AC','SE-Z','SE-K','SE-I']
regioner['Regionkod'] = regionkod
yta=np.array([6524,23800,10968,10559,8190,10437,5427,8504,6075,28029,18118,17519,5118,54665,97239,11165,21549,8424,2931,48935,3135]) #area of region
regioner['Area']=yta
ffall = 9*np.ones(21).astype(int) #week number when first case was recorded
regioner['Första fall'] = ffall
startvecka = ffall - 3

regioner.drop(regioner.index[[20]],inplace=True) #remove Gotland


In [67]:
print(startvecka)
regioner

[6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]


,region,index,population,Regionkod,Area,Första fall
0,Sthlm,8,2389923,SE-AB,6524,9
1,VGR,9,1725881,SE-O,23800,9
2,Skåne,10,1387650,SE-M,10968,9
3,Östergötland,11,467095,SE-E,10559,9
4,Jönköping,12,364750,SE-F,8190,9
5,Södermanland,13,299101,SE-D,10437,9
6,Uppsala,14,387628,SE-C,5427,9
7,Gävleborg,15,287660,SE-X,8504,9
8,Västmanland,16,277074,SE-U,6075,9
9,Örebro,17,305726,SE-T,28029,9


In [68]:
file

,"Nyinskrivna i slutenvård med covid-19, antal patienter per inskrivningsvecka och region",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88
0,Antal unika patienter som rapporterats till So...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Totalt antal inskrivna patienter,NaN,Månadsrapportering till patientregistret,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,vecka 10,NaN,vecka 11,NaN,vecka 12,NaN,vecka 13,...,vecka 48,NaN,vecka 49,NaN,vecka 50,NaN,vecka 51,NaN,vecka 52,NaN
5,NaN,Antal,%*,Antal,%**,Antal,%**,Antal,%**,Antal,...,Antal,%**,Antal,%**,Antal,%**,Antal,%**,Antal,%**
6,Totalt inrapporterat,35057,100,63,0.179707,169,0.482072,639,1.822746,1465,...,1667,4.755113,1568,4.472716,1608,4.586816,1251,3.568474,365,1.041162
7,Rapporterande region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Stockholm,13035,37.182303,37,0.283851,83,0.636747,335,2.570004,734,...,632,4.848485,548,4.204066,521,3.996931,347,2.662064,43,0.329881
9,Västra Götaland,5293,15.098269,9,0.170036,18,0.340072,56,1.058001,94,...,205,3.87304,259,4.893255,323,6.102399,328,6.196864,202,3.816361


In [84]:
from scipy.interpolate import interp1d

def getData(namn,ffall):
    temp=regioner.loc[regioner['region']==namn] #find the right row
    index=temp['index'].values #find the region index
    N=temp['population'].values #find population size of region
    N = int(N[0])
    R = file.iloc[index[0]] 
    R=R.replace(' ','X') #replace missing data with 0
    R=R.replace('X',0) 
    R=R.replace('IU',0) 
    
    ind = np.arange(3,len(R),2) 
    I = R.iloc[ind].values #extract admissions
    #print(I)
    #print(len(I))
    I=I[:-2] #remove last (incomplete) data point
    #print(I)
    padding = np.zeros(10-ffall+3) 
    I=np.insert(I,0,padding) 
    #print(I)
    veckor = np.arange(0,len(I),1) #create array of week numbers
    #print(veckor)
    
    return I,veckor,N


#Load Google mobility data
def loadMobility(namn,svecka):
    df = pandas.read_csv('../data/2020_SE_Region_Mobility_Report.csv') 
    temp=regioner.loc[regioner['region']==namn] 
    kod=temp['Regionkod'].values 
    temp2=df.loc[df['iso_3166_2_code']==kod[0]]
    transit=temp2['transit_stations_percent_change_from_baseline'].values 
    dagar=np.arange(46,46+len(transit))
    tv=np.ceil(dagar/7)
    
    
    gveckor=np.unique(tv)
    k=0
    Tr=np.zeros(int(max(gveckor)-min(gveckor))+1)

    for i in gveckor:
        ind=tv==i
        Tr[k]=np.mean(transit[ind])/100
        k=k+1
    for i in range(1,len(Tr)):
        if np.isnan(Tr[i]):
            Tr[i]=Tr[i-1]
    #print(Tr)
    init=0.2 #set base line travel
    Tr[0:2]=[init,init]
    #print(Tr)
    #print(gveckor)
    #print(gveckor-svecka)
    Trfkn = interp1d(gveckor-svecka,Tr,kind="linear",fill_value="extrapolate")
    #print(Tr)
    return Trfkn

In [89]:
def loadMobility_list(namn_list, svecka):
    
    Trfkn_list = dict()
    for i in namn_list:
       Trfkn_list[i] = loadMobility(i,svecka) 

    return Trfkn_list

In [90]:
namn_list = ['Sthlm','Södermanland']

In [91]:
Trfkn_list = loadMobility_list(namn_list, svecka=6) 

In [92]:
Trfkn_list 

{'Sthlm': <scipy.interpolate._interpolate.interp1d at 0x7e8976968dc0>,
 'Södermanland': <scipy.interpolate._interpolate.interp1d at 0x7e89769686e0>}

In [70]:
df

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-15,2.0,0.0,3.0,4.0,-2.0,0.0
1,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-16,-10.0,-7.0,-30.0,-11.0,-5.0,2.0
2,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-17,-4.0,-3.0,-3.0,-6.0,-6.0,2.0
3,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-18,0.0,0.0,13.0,-4.0,-5.0,1.0
4,SE,Sweden,NaN,NaN,NaN,NaN,NaN,2020-02-19,3.0,1.0,26.0,-3.0,-5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62820,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-11-16,NaN,NaN,NaN,-44.0,-11.0,NaN
62821,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-11-17,NaN,NaN,NaN,-41.0,-13.0,NaN
62822,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-11-18,NaN,NaN,NaN,-43.0,-19.0,NaN
62823,SE,Sweden,Västra Götaland County,Vårgårda Municipality,NaN,NaN,NaN,2020-11-19,NaN,NaN,NaN,-51.0,-19.0,NaN


In [71]:
def beta(t,a,b,transit_fkn):
    return a+b*transit_fkn(t)

def dx(x,t,a,b,lag):
    x1, x2, x3, x4= x[0], x[1], x[2], x[3]
    
    dx1 =  -x1*beta(t-lag,a,b,transit_fkn)*x3/N
    dx2 = x1*beta(t-lag,a,b,transit_fkn)*x3/N - rho*x2
    dx3 = rho*x2 - gamma*x3
    dx4 = gamma*x3
    m = [dx1, dx2, dx3, dx4]
    #m = [np.float64(x[0] if isinstance(i, np.ndarray) else i) for i in m] #made this changes
    return m     
#return [dx1, dx2, dx3, dx4]

#define output from model
def pred_inlagda(t,a,b,lag):
    x0 = [N-1,0,1,0] #initial conditions
    #x0 = [x0[0][0]] + x0[1:]
    #print(x0,t,a,b )

    x = odeint(dx, x0, t,args=(a,b,lag))

    return pv*x[:,2]

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

In [72]:
R = 'Sthlm'
k = 0
I,veckor,N = getData(R,ffall[k])
transit_fkn=loadMobility(R,startvecka[k])

I_train=I
veckor_train=veckor
n=10
ah=np.linspace(3,10,n)
bh=np.linspace(4,10,n)

E=np.zeros((len(ah),len(bh)))
for i in range(n):
    print(i)
    for j in range(n):
        P=pred_inlagda(veckor_train,ah[i],bh[j],delay)
        E[i,j]=np.sqrt(np.sum((P-I_train)**2)/len(veckor_train))

[-0.0025     -0.02       -0.05857143 -0.03428571 -0.25142857 -0.43714286
 -0.45142857 -0.47428571 -0.52142857 -0.42428571 -0.41142857 -0.45
 -0.39285714 -0.39571429 -0.38285714 -0.30571429 -0.32285714 -0.28571429
 -0.33285714 -0.27142857 -0.35       -0.36428571 -0.35142857 -0.35285714
 -0.33714286 -0.30285714 -0.27       -0.27       -0.28142857 -0.28857143
 -0.29428571 -0.26571429 -0.26285714 -0.29714286 -0.30285714 -0.32142857
 -0.31571429 -0.36428571 -0.39142857 -0.41571429 -0.444     ]
[ 0.2         0.2        -0.05857143 -0.03428571 -0.25142857 -0.43714286
 -0.45142857 -0.47428571 -0.52142857 -0.42428571 -0.41142857 -0.45
 -0.39285714 -0.39571429 -0.38285714 -0.30571429 -0.32285714 -0.28571429
 -0.33285714 -0.27142857 -0.35       -0.36428571 -0.35142857 -0.35285714
 -0.33714286 -0.30285714 -0.27       -0.27       -0.28142857 -0.28857143
 -0.29428571 -0.26571429 -0.26285714 -0.29714286 -0.30285714 -0.32142857
 -0.31571429 -0.36428571 -0.39142857 -0.41571429 -0.444     ]
0
1
2
3
4
5


In [73]:
ah

array([ 3.        ,  3.77777778,  4.55555556,  5.33333333,  6.11111111,
        6.88888889,  7.66666667,  8.44444444,  9.22222222, 10.        ])

In [74]:
optind = np.where(E == np.amin(E))
afit = ah[optind[0]]
bfit = bh[optind[1]]

print("Minimum RMSE: ",E[optind])
print("a=",afit,"b=",bfit)

Minimum RMSE:  [216.66060986]
a= [4.55555556] b= [8.66666667]


In [75]:
optind 

(array([2]), array([7]))

In [76]:
regs = pandas.read_pickle('../data/param_IC.pkl') #load initial guesses for a and b for each region
astart = regs['a'].values
bstart = regs['b'].values

In [77]:
regs

,region,index,population,Regionkod,a,b,E
0,Sthlm,8,2389923,SE-AB,4.759494,9.215190,175.235300
1,VGR,9,1725881,SE-O,3.848101,9.493671,90.821751
2,Skåne,10,1387650,SE-M,3.620253,8.240506,41.357155
3,Östergötland,11,467095,SE-E,3.734177,8.379747,17.383155
4,Jönköping,12,364750,SE-F,3.278481,10.050633,21.330193
5,Södermanland,13,299101,SE-D,3.734177,9.215190,15.531288
6,Uppsala,14,387628,SE-C,3.506329,6.430380,17.883654
7,Gävleborg,15,287660,SE-X,3.164557,10.746835,20.700678
8,Västmanland,16,277074,SE-U,3.506329,6.848101,8.616789
9,Örebro,17,305726,SE-T,3.392405,8.379747,12.290108
